## Waste Detection With YOLOv4
This notebook contains instructions on training YOLOv4 to perform object detection on the [ZeroWaste](https://github.com/dbash/zerowaste) dataset. Our goal is to train a customized YOLOv4 network to efficiently and accurately identify different waste objects in a setting that highly resembles environment of a real waste recycle center.  
Our implementation is based on the PyTorch framework. We borrowed the yolov4 base model from [here](https://github.com/WongKinYiu/PyTorch_YOLOv4). The instructions provided in this notebook also applies to other PyToch YOLO implementations authored by WongKinYiu.  

__Important Note__: The commands given in this notebook is recommended to be run in an actual terminal as some commands may print very long logs.

### 1. Environment Setup
1. __Connecting to SCC__  
Please the README file in our repo for instructions on how to connect to an SCC server and start jupyter lab on the SCC.  
2. __Load necessary modules__    
type the following command to load necessary modules on the SCC
    ```
    module load cuda/10.1
    module load python3/3.8.10
    ```


### 2. Getting Ready for Training
Before we train the network, there are several configurations we need to do.

__Installing Dependencies__:  
The PyTorch version of the YOLOv4 network is already cloned to our project repo on the SCC. The absolute path for the repo is `/projectnb/dl523/projects/RWD`. We need to cd into that directory to run the cells in this notebook. To do that, run the following command:

In [ ]:
%cd ./PyTorch_YOLOv4

To run the YOLOv4 network, we need to install some packages, run the following cell to install the necessary packages

In [ ]:
!pip install -r requirements.txt

After installing the pakcages needed to run the model, we need to do some preprocessing on our dataset. The YOLOv4 model requires the train/val/test data to follow a specific file structure and format. We used [Roboflow](https://roboflow.com/convert/coco-json-to-yolov5-pytorch-txt) to convert our dataset from coco format to YOLOv4/5 format. To pull the converted dataset folder from roboflow, simply run 
```
python tools/download_dataset.py
```
 This script will pull the correct data in YOLO compatible format to the project directory

### 3. Running YOLOv4

YOLOv4 is a highly customizable model; the model is constructed according to the configuration file provided. The config file specifies the network architectures as well as the hyperparameters used in the network. When running the network, we can also pass in different arguments to specify how we want to train the network. Some available arguments for training are as follows:
- `--weights` This argument specifies which weight file we are using when running the network. (we need to download some pre-trained weights)
- `--cfg` This argument specifies which config file we are using to construct the model
- `--data` This argument specifies which data we are using. We pass in the path to the data.yaml file, which contains our dataset information
- `--hyp` This argument specifies the customized hyperparameter we are using. It's usually used after we've trained a network for a while, and we want to continue training using the updated hyperparameters instead of the default, initial hyperparameters
- `--epochs` This argument specifies the number of training epochs the model will go through
- `--batch-size` This argument specifies the batch size of the training data
- `--img-size` This argument specifies the size of the training/test images. The values are in the format 'height width', and each must be a multiple of 64
- `--resume` This argument denotes whether we want to resume training from a specific checkpoint of our last run. The default value for this argument is False. If we want to resume training using a specific weight file, pass a string containing the path to that weight file ending in `.pt` as the value of this argument. If we simply want to train from the very last checkpoint, pass True as the value of this argument.
- `--nosave` This argument tells the network to only stores the very last checkpoint information of the training. The default value for this argument is True.
- `--notest` This argument tells the network to only test the model performance at the last epoch of the training. The default value for this argument is True. 
- `--evolve` This argument specifies whether we want to evolve the hyperparameters during training. The default value for this argument is True.
- `--cache-images` This argument allows the model to cache images for faster training. The default value for this argument is True
- `--image-weights` This argument allows the model to use weighted image selection for training.
- `--device` This argument specifies the device the model will be trained on. The value can be 0, 1, 2, 3, or cpu.
- `--adam` This argument specifies that the model will use Adam optimizer. 
- `--workers` This argument specifies the maximum number of dataloader workers
- `--project` This arugment specifies the project directory the results will be saved to. The default directory is `runs/train`
- `--name` This argument specifies the name of the current run. Ex. yolov4-csp-x-leaky. The result of the run will be saved to `project/<name>`

There are other available arguments not documented here, please check `train.py` for their usage

To train YOLOv4, use the following command: (It's recommended to run this command directly in the terminal)

In [ ]:
!python train.py --device 1 --batch-size 16 --img 640 640 --data zero-waste-1/data.yaml --cfg cfg/yolov4-csp-x-leaky.cfg --weights weights/yolov4-csp-x-leaky.weights --name yolov4-csp-x-leaky-300-ep --epochs 300

We can pass in other aguments and change existing arguments as needed. By default, the above command will run the network for 300 epochs. After the network finished running all epochs, all the parameters and results are saved to the `run` directory. The graph of model performance measurements can be viewed through `wandb` if you have a wandb account. The graphs include measurements like mAP etc. 

After running the training procedure, the result and final weights of the model are stored in `runs/train/<name>` folder (name is the name argument passed into train.py). The `results.txt` file will show you the intermediate result at each epoch. The result will be similar to the following table:

|Epoch|gpu_mem|box_loss|obj_loss|cls_loss|total_loss|targets|img_size|precision|recall|mAP@.5|mAP@.5:.95|x/lr0|x/lr1|x/lr2|
|:---:|:-----:|:------:|:------:|:------:|:--------:|:-----:|:------:|:-------:|:----:|:----:|:--------:|:---:|:---:|:---:|
| 0/9 | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... |
| ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... |
| 8/9 | 15G | 0.05565 | 0.05123 | 0.001666| 0.1085 | 92 | 640 | 0.4066 | 0.09914 | 0.1032 | 0.05358 | 0.04567 | 0.03779 | 0.001744 |

After training, we can used the trained model to do inference. To use the trained model, we first need to locate the trained weights. The weights are located at `runs/train/<name>/weights` directory. It would be best to copy the saved weight file to the `weights` folder under the project root directory for easy access. The `test.py` file also accepts some arguments:  
- `--weights` The path to the trained weights file we are using to do inference
- `--data` The path to the data file that contains information of our dataset
- `--batch-size` Thie argument specifies the batch of the image size
- `--img-size` This argument specifies the image inference size
- `--conf-thres` This argument specifies the object confidence threshold
- `--iou-thres` This argument specifies the IoU threshold for NMS
- `--task` This argument specifies the type of task we are performing with `test.py`. We can choose `val`, `test`, or `study`.
- `--device` This argument specifies the device we are using when running `test.py`. The value can be 0, 1, 2, 3, or cpu.
- `--single-cls` This argument makes the model treat the dataset as a single class dataset
- `--augment` This argument enables the model to perform augmented inference
- `--verbose` This argument makes the model report mAP by each class
- `--save-txt` This argument allows for saving inference result of each image in text files
- `--save-conf` This argument allows for saving confidences in the text file generated using `--save-txt` argument
- `--save-json` This argument allows for saving a cocoapi-compatible JSON results file
- `--project` This argument specifies the project directory the results will be saved to. The default directory is `runs/test`
- `--name` This argument specifies the name of the current run. Ex. yolov4-csp-x-leaky. The result of the run will be saved to `project/<name>`
- `--cfg` This argument specifies which config file we are using to construct the model
- `--names` This argument specifies the class name file for our dataset. A class name file is in the form `<dataset name>.names`

To test a specific model, use the command below and modify the arguments as needed.

In [ ]:
!python test.py --img-size 640 --conf-thres 0.001 --batch 16 --device 0 --data zero-waste-1/data.yaml --cfg cfg/yolov4-csp-x-leaky.cfg --weights weights/best.pt --task test --names zero-waste-1/zero-waste.names --verbose
